In [129]:
# Dependencies
import time
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import requests


In [130]:
# Setup chrome driver for Chrome tool
def init_browser():
    #executable_path = {"executable_path": "C:/chromedriver/chromedriver"}
    executable_path = {'executable_path': ChromeDriverManager().install()}
    return Browser("chrome", **executable_path, headless=False)

In [131]:
# ------------------------------PART 1 START-------------------------------#

# Setup splinter
browser = init_browser()
#browser = Browser('chrome', **executable_path, headless=False)

# URL of page to be scraped
news_url = "https://mars.nasa.gov/news/"

browser.visit(news_url)

# To stop errors because it took a second to load
time.sleep(1)

# Store the html object
news_html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
mars_news_soup = BeautifulSoup(news_html, 'html.parser')

# Grab the whole list then rip it apart for easier access and less prone to errors
mars_news_list = mars_news_soup.find('ul', class_='item_list')

latest_headline = mars_news_list.find('li', class_ = 'slide')

news_title = latest_headline.find("div", class_='content_title').text

news_content = mars_news_soup.find("div", class_='article_teaser_body').text

# Close Browser.
browser.quit()

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [C:\Users\slobanwala\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


<ul class="item_list"><li class="slide"><div class="image_and_description_container"><a href="/news/8815/from-jpls-mailroom-to-mars-and-beyond/" target="_self"><div class="rollover_description"><div class="rollover_description_inner">Bill Allen has thrived as the mechanical systems design lead for three Mars rover missions, but he got his start as a teenager sorting letters for the NASA center.</div><div class="overlay_arrow"><img alt="More" src="/assets/overlay-arrow.png"/></div></div><div class="list_image"><img alt="Bill Allen standing in the Mars yard" src="/system/news_items/list_view_images/8815_C1-bill-allen-Mars-Yard-2020-320.jpg"/></div><div class="bottom_gradient"><div><h3>From JPL's Mailroom to Mars and Beyond</h3></div></div></a><div class="list_text"><div class="list_date">December 14, 2020</div><div class="content_title"><a href="/news/8815/from-jpls-mailroom-to-mars-and-beyond/" target="_self">From JPL's Mailroom to Mars and Beyond</a></div><div class="article_teaser_bod

In [132]:
# Retrieve page with the requests module
#response = requests.get(url)
print(news_title + '\n' + news_content)
# ------------------------------PART 1 END-------------------------------#

From JPL's Mailroom to Mars and Beyond
Bill Allen has thrived as the mechanical systems design lead for three Mars rover missions, but he got his start as a teenager sorting letters for the NASA center.


In [133]:
# ------------------------------PART 2 START-------------------------------#
# JPL Mars Space Images - Featured Image
# Setup splinter
browser = init_browser()

jpl_mars_img_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_mars_img_url)

# To stop errors because it took a second to load
time.sleep(1)

# https://splinter.readthedocs.io/en/latest/api/driver-and-element-api.html
# use click_link_by_partial_text
browser.click_link_by_partial_text("FULL IMAGE")

# To stop errors because it took a second to load
time.sleep(1)

# Have to click more info to open up bigger image, use find_link_by_partial_text again. Really useful function
browser.is_element_present_by_text("more info", wait_time=1)
bigger_img_elem = browser.find_link_by_partial_text("more info")
bigger_img_elem.click()

# To stop errors because it took a second to load
time.sleep(1)

# Store the html object
img_html = browser.html

# Time to strip the image and store
mars_img_soup = BeautifulSoup(img_html, 'html.parser')

mars_img_url = mars_img_soup.find("figure", class_="lede").a["href"]

featured_image_url = f'https://www.jpl.nasa.gov{mars_img_url}'

# Close Browser.
browser.quit()

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\slobanwala\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


C:\Users\slobanwala\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


In [140]:
print(featured_image_url)
# ------------------------------PART 2 END-------------------------------#

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18847_hires.jpg


In [135]:
# ------------------------------PART 3 START-------------------------------#
# Mars Facts
# Much easier since just scraping a table, and we did a ton of that in the simpsons analysis for ETL project
mars_facts_url = "https://space-facts.com/mars/"

# use read_html
mars_facts_tables = pd.read_html(mars_facts_url)

mars_facts_df = mars_facts_tables[0]

# Set first column to Description as the prompt shows, and the second is Measurement/Units
mars_facts_df.columns = ['Description', 'Measurement/Units']

mars_facts_df

,Description,Measurement/Units
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [142]:
# To use on website we need it in html format
# Remove header just use html to produce header as it looks nicer
mars_facts_df.to_html('Resources/mars_facts.html', index=False, header=False)
mars_facts = mars_facts_df.to_html(index=True, header=True)
print(mars_facts)
# ------------------------------PART 3 END-------------------------------#

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Measurement/Units</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th

In [137]:
# ------------------------------PART 4 START-------------------------------#
# Mars Hemispheres
browser = init_browser()

mars_hemis_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

mars_hemis_homepage = "https://astrogeology.usgs.gov";

browser.visit(mars_hemis_url)

# To stop errors because it took a second to load
time.sleep(1)

# store into html object again
hemis_html = browser.html

hemis_soup = BeautifulSoup(hemis_html, 'html.parser')

# Grabs all 4 divs of the hemispheres
hemis_url = hemis_soup.find_all("div", class_="item")

# Store them as seperate html objects
hemis_url_all = []

for hemisphere in hemis_url:
    curr_hemis_url = hemisphere.find('a')['href']
    hemis_url_all.append(curr_hemis_url)

# Close Browser.
browser.quit()

print(hemis_url_all)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\slobanwala\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


 
['/search/map/Mars/Viking/cerberus_enhanced', '/search/map/Mars/Viking/schiaparelli_enhanced', '/search/map/Mars/Viking/syrtis_major_enhanced', '/search/map/Mars/Viking/valles_marineris_enhanced']


In [138]:
# Go through each link and store title and full link in a dict
mars_hemis_urls = []

for hemisphere in hemis_url_all:
    
    # create full link
    mars_url_full = mars_hemis_homepage + hemisphere
    
    # Abuse splinter again
    browser = init_browser()
    browser.visit(mars_url_full)
    
    # To stop errors because it took a second to load
    time.sleep(1)
    
    hemis_html = browser.html
    hemis_soup = BeautifulSoup(hemis_html, 'html.parser')
    
    # Grab title
    hemis_title = hemis_soup.find("h2", class_="title").text
    
    # Grab the full.jpg for all the Hemispheres, aka the large size
    hemis_img_url = hemis_soup.find("li").a['href']
    
    mars_hemis_urls.append({'title': hemis_title, 'img_url': hemis_img_url})
    
    # Close Browser.
    browser.quit()

print(mars_hemis_urls)
# ------------------------------PART 4 END-------------------------------#

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\slobanwala\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\slobanwala\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\slobanwala\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\slobanwala\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


 
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [139]:
# ------------------------------PART 5 START-------------------------------#
# Step 2 - MongoDB and Flask Application
mission_to_mars_data = {}

# Store all data
mission_to_mars_data['news_title'] = news_title
mission_to_mars_data['news_content'] = news_content
mission_to_mars_data['featured_image_url'] = featured_image_url
mission_to_mars_data['mars_facts'] = mars_facts
mission_to_mars_data['mars_hemisphere_urls'] = mars_hemis_urls

print(mission_to_mars_data)

{'news_title': "From JPL's Mailroom to Mars and Beyond", 'news_content': 'Bill Allen has thrived as the mechanical systems design lead for three Mars rover missions, but he got his start as a teenager sorting letters for the NASA center.', 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18847_hires.jpg', 'mars_facts': '<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </

In [113]:
# ------------------------------PART 5 END-------------------------------#